In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor

dfTrain = pd.read_csv('../../../DataSet/trip_train.csv', parse_dates=['start_date', 'end_date'], infer_datetime_format=True)
dfTest = pd.read_csv('../../../DataSet/trip_test.csv', parse_dates=['start_date', 'end_date'], infer_datetime_format=True)
dfWeather = pd.read_csv('../../../DataSet/weather.csv', parse_dates=['date'], infer_datetime_format=True)
dfStation = pd.read_csv('../../../DataSet/station.csv')

In [2]:
dfTrain = dfTrain[['id', 'duration', 
                   'start_date', 'start_station_id', 
                   'subscription_type']]

dfTest = dfTest[['id', 
                 'start_date', 
                 'start_station_id', 
                 'subscription_type']]

dfTrain['Order'] = pd.Series(xrange(len(dfTrain)))
dfTest['Order'] = pd.Series(xrange(len(dfTest)))

dfWeather = dfWeather[['date', 'zip_code', 'mean_temperature_f']]

print len(dfTrain)
dfTrain.head()

549961


,id,duration,start_date,start_station_id,subscription_type,Order
0,907649,396,2015-08-27 08:36:00,50,Subscriber,0
1,384043,636,2014-07-28 22:06:00,67,Subscriber,1
2,316176,334,2014-06-09 08:42:00,77,Subscriber,2
3,618874,666,2015-01-26 16:55:00,69,Subscriber,3
4,910977,318,2015-08-29 15:09:00,67,Subscriber,4


In [3]:
dates = {'year':dfTrain.start_date.dt.year,
         'month':dfTrain.start_date.dt.month, 
         'day':dfTrain.start_date.dt.dayofyear, 
         'weekday':dfTrain.start_date.dt.dayofweek, 
         'hour':dfTrain.start_date.dt.hour}

wdates = {'year':dfWeather.date.dt.year, 
          'month':dfWeather.date.dt.month, 
          'day':dfWeather.date.dt.dayofyear}

dfTrain = dfTrain.join(pd.DataFrame(dates))

dates = {'year':dfTest.start_date.dt.year, 
         'month':dfTest.start_date.dt.month, 
         'day':dfTest.start_date.dt.dayofyear, 
         'weekday':dfTest.start_date.dt.dayofweek, 
         'hour':dfTest.start_date.dt.hour}

dfTest = dfTest.join(pd.DataFrame(dates))

subscriptionTypes = dfTrain.subscription_type.unique()
dfTrain.subscription_type = dfTrain.subscription_type.astype('category', categories=subscriptionTypes).cat.codes
dfTest.subscription_type = dfTest.subscription_type.astype('category', categories=subscriptionTypes).cat.codes

In [4]:
dfWeather = dfWeather.join(pd.DataFrame(wdates))[['year', 'month', 'day', 'zip_code',
                                                  'mean_temperature_f']]
dfWeather.head()

,year,month,day,zip_code,mean_temperature_f
0,2013,8,241,94107,68.0
1,2013,8,242,94107,69.0
2,2013,8,243,94107,64.0
3,2013,9,244,94107,66.0
4,2013,9,245,94107,69.0


In [5]:
dfTrain = dfTrain[['Order', 'id', 'duration', 'year', 
                   'month', 'weekday', 'day', 
                   'hour', 'start_station_id', 
                   'subscription_type']]

dfTest = dfTest[['Order', 'id', 'year', 'month', 
                 'weekday', 'day', 
                 'hour', 'start_station_id', 
                 'subscription_type']]

dfTrain.head()

,Order,id,duration,year,month,weekday,day,hour,start_station_id,subscription_type
0,0,907649,396,2015,8,3,239,8,50,0
1,1,384043,636,2014,7,0,209,22,67,0
2,2,316176,334,2014,6,0,160,8,77,0
3,3,618874,666,2015,1,0,26,16,69,0
4,4,910977,318,2015,8,5,241,15,67,0


In [6]:
def cityNameToZipCode(row) :

    if row.city == 'San Francisco' :
        return 94107

    if row.city == 'Redwood City' :
        return 94063

    if row.city == 'Palo Alto' :
        return 94301

    if row.city == 'Mountain View' :
        return 94041

    if row.city == 'San Jose' :
        return 95113


In [7]:
# Preparo el STATION dataframe
dfStation.loc[:,'city'] = dfStation.apply(cityNameToZipCode, axis=1)
dfStation.rename(columns={'id':'start_station_id', 'city':'zip_code'}, inplace=True)
dfStation = dfStation[['start_station_id', 'zip_code']]



In [8]:
dfTrain = dfTrain.merge(dfStation, on='start_station_id')[['Order', 'id', 'duration', 
                                                           'start_station_id', 'subscription_type', 
                                                           'weekday', 'year', 'month', 'day', 
                                                           'hour', 'zip_code']]

dfTest = dfTest.merge(dfStation, on='start_station_id')[['Order', 'id',
                                                           'start_station_id', 'subscription_type', 
                                                           'weekday', 'year', 'month', 'day', 
                                                           'hour', 'zip_code']]

In [9]:
dfTrain = dfTrain.merge(dfWeather, on=['year', 'month', 'day', 'zip_code'])
dfTest = dfTest.merge(dfWeather, on=['year', 'month', 'day', 'zip_code'])

In [10]:
print len(dfTrain)
dfTrain.head()

549961


,Order,id,duration,start_station_id,subscription_type,weekday,year,month,day,hour,zip_code,mean_temperature_f
0,0,907649,396,50,0,3,2015,8,239,8,94107,72.0
1,3700,907916,981,50,0,3,2015,8,239,9,94107,72.0
2,11801,908620,585,50,0,3,2015,8,239,17,94107,72.0
3,11876,907429,1772,50,0,3,2015,8,239,7,94107,72.0
4,12703,907759,510,50,0,3,2015,8,239,8,94107,72.0


In [11]:
# Elimino las rows sin muestreo de temperatura
dfTrain = dfTrain.loc[~dfTrain.mean_temperature_f.isnull()]
dfTrain.sort_values(by='Order', inplace=True)
dfTrain.head()

,Order,id,duration,start_station_id,subscription_type,weekday,year,month,day,hour,zip_code,mean_temperature_f
0,0,907649,396,50,0,3,2015,8,239,8,94107,72.0
245503,1,384043,636,67,0,0,2014,7,209,22,94107,67.0
192095,2,316176,334,77,0,0,2014,6,160,8,94107,68.0
220417,3,618874,666,69,0,0,2015,1,26,16,94107,51.0
433359,4,910977,318,67,0,5,2015,8,241,15,94107,71.0


In [12]:
def setSFMeanTemperature(row) :
    if np.isnan(row.mean_temperature_f) :
        month = row.month
        hour = row.hour
        temps = [50.0, 52.5, 54.5, 56.5, 56.5, 59.0, 66.0, 64.5, 62.5, 61.0, 56.5, 50.5]
        
dfTest[dfTest.mean_temperature_f.isnull()]

,Order,id,start_station_id,subscription_type,weekday,year,month,day,hour,zip_code,mean_temperature_f
114550,65241,881579,34,1,5,2015,8,220,16,94301,NaN
114551,112898,881570,34,1,5,2015,8,220,16,94301,NaN
114552,442,881480,35,1,5,2015,8,220,14,94301,NaN
115511,4870,749986,32,0,4,2015,5,121,18,94041,NaN
115512,115949,749997,32,0,4,2015,5,121,18,94041,NaN
115513,5863,750167,29,1,4,2015,5,121,22,94041,NaN
115514,32478,748767,28,0,4,2015,5,121,8,94041,NaN
115515,73490,748632,33,0,4,2015,5,121,7,94041,NaN
115516,29446,749494,27,0,4,2015,5,121,15,94041,NaN
115517,17191,748816,30,0,4,2015,5,121,8,94041,NaN


In [31]:
np.isnan(dfTest.loc[114550].mean_temperature_f)

True